## <b>M5_종합실습</b>

### 1.정형(테이블) 데이터 가져오기

> 데이터 원본 종류별 적용 패키지
- CSV : pandas
- 웹 사이트 데이터 : selenium + bs4 + Pandas

> 최종 결과 데이터 형태 : Pandas DataFrame 
- 데이터를 가져오면,
- 데이터 프레임 형태의 데이터가 결과물로 반환 되는데,
- 이후, 결과 데이터를 활용하여 통계 작업 등에 활용

#### 1) CSV 파일 가져오기

> 데이터셋 : data_set/sample_table.csv (공공데이터포털 - 과학기술정보통신부 중앙전파관리소_국내외 신규위성 발사 현황_20230930)

In [ ]:
import pandas as pd

df = pd.read_csv('data_set/sample_table.csv', encoding='euc-kr')
df

#### 2) 웹 데이터 가져오기

> 데이터셋 : http://new-collar.kr/data_set/sample_table.html (공공데이터포털 - 과학기술정보통신부 중앙전파관리소_국내외 신규위성 발사 현황_20230930)

In [ ]:
url = 'http://new-collar.kr/data_set/sample_table.html'

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import pandas as pd

driver = webdriver.Chrome()

driver.get(url)
driver.implicitly_wait(15)

soup = bs(driver.page_source, 'html.parser')

table = '#personal_profile' # id값으로 가져오기
info = soup.select(table) # select로 가져오기

table = pd.read_html(str(info)) # pd.read_html으로 읽기 위해 str로 변환
df = table[0] # 리스트 형식의 table 내에 데이터 프레임 1개만 존재하여 table[0]으로 추출
df

### 2.정형(테이블) 데이터 분석

> 기본 정보 확인하기

In [ ]:
df.info()

> 기본 통계

In [ ]:
df.describe()

> 총 발사실패 건수

In [ ]:
df['발사실패'].sum()

> 항목별 합계

In [ ]:
df.sum(axis=0, numeric_only=True)

> 년도별 발사 건수

In [ ]:
df['년도'] = df['연월일'].str[:4] # '년도'만 추출하여 열 추가
df.groupby('년도')['정지위성'].sum() # '년도'로 그룹화하여 '정지위성' 합계

### 3.비정형(텍스트) 데이터 가져오기

#### 1) 텍스트 파일 가져오기 - read()를 활용

> 데이터셋 :  data_set/sample_text.txt  (wikipedia.org - Society)

In [ ]:
f = open('data_set/sample_text.txt', 'r', encoding='utf-8') 
raw = f.read() # 모든 텍스트를 텍스트 형식으로 반환
raw[:1000]

- 데이터 형식 확인

In [ ]:
type(raw)

- 문자열 길이 확인

In [ ]:
len(raw)

#### 2) 텍스트 파일 가져오기 - readlines()를 활용

> 데이터셋 : data_set/sample_text.txt  (wikipedia.org - Society)

In [ ]:
f = open('data_set/sample_text.txt', 'r', encoding='utf-8') 
raw = f.readlines() # 개행문자를 기준으로 라인 단위 리스트 형식로 반환
print(raw[:3])

- 데이터 형식 확인

In [ ]:
type(raw)

- 문자열 길이 확인

In [ ]:
len_str = 0

for r in raw:
    len_str += len(r)
len_str

#### 3) 웹 데이터 가져오기

> 데이터셋 : http://new-collar.kr/data_set/sample_text.html  (wikipedia.org - Society)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

url = 'http://new-collar.kr/data_set/sample_text.html'

driver = webdriver.Chrome()
driver.get(url)

driver.implicitly_wait(15)

raw = driver.find_element(By.XPATH, '/html/body/main/div[2]/div/p[2]').text # 텍스트만 추출
raw[:1000]

- 데이터 형식 확인

In [ ]:
type(raw)

- 문자열 길이 확인

In [ ]:
len(raw) # 텍스트 파일과 웹 데이터 추출 데이터의 길이가 다를 수 있음

#### 4) 한글 텍스트 파일 가져오기 - read()를 활용

> 데이터셋 :  : data_set/sample_text_ko.txt  (wikipedia.org - 사회)

In [ ]:
f = open('data_set/sample_text_ko.txt', 'r', encoding='utf-8') 
raw_ko = f.read() # 모든 텍스트를 텍스트 형식으로 반환
raw_ko[:1000]

- 데이터 형식 확인

In [ ]:
type(raw_ko)

- 문자열 길이 확인

In [ ]:
len(raw_ko)

### 4.비정형(텍스트) 데이터 분석

#### 1) 키워드 빈도 분석(한글)

> 형태소 분석(토큰화)

In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

kiwi = Kiwi()

- 기본 토큰화

In [ ]:
tokens = kiwi.tokenize(raw_ko)
print(tokens[:10])
print(len(tokens)) # 토큰 갯수 확인

- 키워드(단어)만 추출

In [ ]:
tokens = kiwi.tokenize(raw_ko)
token_text = []

for token in tokens:
    token_text.append(token.form) # 형태소만 append
      
print(token_text[:100])
print(len(token_text)) # 토큰 갯수 확인

- 명사만 추출

In [ ]:
tokens = kiwi.tokenize(raw_ko)
token_text = []

for token in tokens:
    if token.tag[0] == 'N': # 명사인 경우만
        token_text.append(token.form) #형태소만 append
      
print(token_text[:100])
print(len(token_text)) # 토큰 갯수 확인

- 불용어 제거 : kiwipiepy 내장 불용어 사용

In [ ]:
stopwords = Stopwords()

tokens = kiwi.tokenize(raw_ko, stopwords=stopwords) # 내장 불용어 제외 후 토큰화
print(tokens[:10])
print(len(tokens)) # 토큰 갯수 확인

- 불용어 제거 : 사용자 정의 불용어 사용

In [ ]:
user_stop = ['사회', '영어']

tokens = kiwi.tokenize(raw_ko)
token_text = []

for token in tokens:
    if token.form not in user_stop: # 형태소가 불용어가 아니라면
        token_text.append(token.form) # 형태소만 append
      
print(token_text[:100])
print(len(token_text)) # 토큰 갯수 확인

- 여러 조건 적용 : 명사만, 내장+사용자 정의 불용어 제외

In [ ]:
stopwords = Stopwords()
user_stop = ['사회', '영어']

tokens = kiwi.tokenize(raw_ko, stopwords=stopwords) # 내장 불용어 제외 후 토큰화
token_text = []

for token in tokens:
    if (token.tag[0] == 'N') & (token.form not in user_stop): # 명사만, 사용자 불용어 제외
        token_text.append(token.form) #형태소만 append
      
print(token_text[:100])
print(len(token_text)) # 토큰 갯수 확인

> 단어 빈도수 측정

In [ ]:
from collections import Counter

c = Counter(token_text)
c

- 빈도수 높은 10개 추출

In [ ]:
top_c = c.most_common(10) 
print(top_c)

#### 2) BoW로 빈도 분석(영어)

> 텍스트 파일에서 가져오기

In [ ]:
f = open('data_set/sample_text.txt', 'r', encoding='utf-8') 
raw = f.read() # 모든 텍스트를 텍스트 형식으로 반환
print(type(raw))
print(raw[:1000])

> BoW 만들기

- 내장 불용어 제외

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer(stop_words='english')

text = [raw] # 리스트 형식으로 변환
bow = vector.fit_transform(text)

print(vector.get_feature_names_out()) # 토큰화된 단어 목록 확인
print(bow.toarray()) # 배열 형태로 빈도수 출력

- 사용자 불용어 제외

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

user_stop = ['000', '01']
vector = CountVectorizer(stop_words=user_stop)

text = [raw] # 리스트 형식으로 변환
bow = vector.fit_transform(text)

print(vector.get_feature_names_out()) # 토큰화된 단어 목록 확인
print(bow.toarray()) # 배열 형태로 빈도수 출력

- 내장+사용자 불용어 제외

In [ ]:
# 내장 불용어 확인

from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer(stop_words='english')
print(vector.get_stop_words())

In [ ]:
# 내장 불용어를 리스트로 변환
s_w = list(vector.get_stop_words())
print(s_w)

In [ ]:
# 내장 불용어(stopwords)에 사용자 불용어(user_stop) 추가

s_w.extend(user_stop)
print(s_w)

vector = CountVectorizer(stop_words=s_w)
bow = vector.fit_transform(text)

print(vector.get_feature_names_out()) # 토큰화된 단어 목록 확인
print(bow.toarray()) # 배열 형태로 빈도수 출력

> DTM에서 키워드별 빈도수 확인하기

In [ ]:
import pandas as pd

# columns = []
# for k, v in sorted(vector.vocabulary_.items(), key=lambda item:item[1]):
#       columns.append(k)

# df = pd.DataFrame(bow.toarray(), columns=columns)

df = pd.DataFrame(bow.toarray(), columns=vector.get_feature_names_out())
df

- 내림차순 정렬

In [ ]:
df.sort_values(by=0, axis=1, ascending=False)

- 상위 10개 추출

In [ ]:
df.sort_values(by=0, axis=1, ascending=False).iloc[:,:10]

> 키워드별 인덱스 확인하기

In [ ]:
vector.vocabulary_

- 인덱스로 정렬

In [ ]:
vocab = vector.vocabulary_
sorted(vocab.items(), key=lambda x:x[1])

- 키워드로 인덱스 찾기

In [ ]:
vocab['technology']

In [ ]:
key = ['technology', '15', '2000']
value = []
for k in key:
    value.append(vector.vocabulary_[k])
value 